In [ ]:
from yahist import Hist1D, Hist2D
import json
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats as st

In [ ]:
df = ak.from_parquet('../output_analysis_data/tot_200/start_1948_stop_2086_bias_285_offset_15.0_energy_5.0_power_i1_module_37_LP2_20_cubicLM_unbinned/fullCollection.parquet')

In [ ]:
tots = ak.flatten(df.tot)
tot_codes = ak.flatten(df.tot_code)
toas = ak.flatten(df.toa)
toa_codes = ak.flatten(df.toa_code)
rows = ak.flatten(df.row)
cols = ak.flatten(df.col)
dts = ak.flatten(df.dt)
cal_codes = ak.flatten(df.cal_code)

In [ ]:
tot_lows = [50]#,80,90]
tot_highs = [200]#,120,130,140]
toa_lows = [20]
toa_highs = [600]

tot_low=50
tot_high=200
toa_low=20
toa_high=600

In [ ]:
f = plt.figure()
f.set_figwidth(9)
f.set_figheight(6)

row=7
col=10
dt_bins=np.linspace(11,14,50)
px_mask = (rows==row) & (cols==col)

# Find the cal code selections for every toa/tot combo
#tot_sel= (tot_codes>tot_low) & (tot_codes<tot_high)
#toa_sel= (toa_codes>toa_low) & (toa_codes<toa_high)
#data_mask = tot_sel & toa_sel
cal_mode = st.mode(np.array(cal_codes[px_mask]))[0]
cal_mask = (cal_codes>cal_mode-2) & (cal_codes<cal_mode+2)
px_tot_codes = tot_codes[px_mask & cal_mask]
px_toa_codes = toa_codes[px_mask & cal_mask]
px_dts = dts[px_mask & cal_mask]

# Cal only cuts
h1 = Hist1D(px_dts,bins=dt_bins,label=f'no cut events {len(px_dts)}')
h1.plot()
h1.fit('gaus',color='r',label='no_cut')
plt.title('Resolution vs cuts (no cut)')
plt.xlabel('dt')
plt.ylabel('count')
#plt.savefig(f'DESY_tot_toa_studies/images/dt_no_cut_r{row}c{col}.png')
for tot_low in tot_lows:
    for tot_high in tot_highs:
        # Only cutting on tot
        plt.clf()
        tot_mask = (px_tot_codes>tot_low) & (px_tot_codes<tot_high)
        h2 = Hist1D(px_dts[tot_mask],
                        bins=dt_bins,
                        label='tot cut events {} ({:.1f}%)'.format(ak.sum(tot_mask),100*ak.sum(tot_mask)/len(px_dts)))
        h2.plot()
        h2.fit('gaus',color='r',label='tot_cut')
        plt.title('Resolution vs cuts (tot cut)')
        plt.xlabel('dt')
        plt.ylabel('count')
        #plt.savefig(f'DESY_tot_toa_studies/images/dt_tot_cut_r{row}c{col}_tot_low{tot_low}_tot_high{tot_high}.png')

        for toa_low in toa_lows:
            for toa_high in toa_highs:
                # Scatter plot
                plt.clf()
                plt.scatter(px_tot_codes,px_toa_codes,label=f'total events: {len(px_dts)}',color='black')
                plt.xlabel('tot code')
                plt.ylabel('toa code')
                plt.title(f'TOA Code vs TOT Code row:{row} col:{col}')
                plt.legend()
                plt.axvline(tot_low,color='r')
                plt.axvline(tot_high,color='r')
                plt.axhline(toa_low,color='g')
                plt.axhline(toa_high,color='g')
                #plt.savefig(f'DESY_tot_toa_studies/images/toa_v_tot_scatter_r{row}c{col}_tot_low{tot_low}_tot_high{tot_high}_toa_low{toa_low}_toa_high{toa_high}.png')

                # Toa and Tot cuts
                plt.clf()
                toa_mask = (px_toa_codes>toa_low) & (px_toa_codes<toa_high)
                h4 = Hist1D(px_dts[tot_mask & toa_mask],
                            bins=dt_bins,
                            label='tot and toa cut events {} ({:.1f}%)'.format(ak.sum(tot_mask&toa_mask),100*ak.sum(tot_mask&toa_mask)/len(px_dts)))
                h4.plot()
                h4.fit('gaus',color='r',label='toa_cut')
                plt.title('Resolution vs cuts (tot and toa cut)')
                plt.xlabel('dt')
                plt.ylabel('count')
                #plt.savefig(f'DESY_tot_toa_studies/images/dt_tot_and_toa_cut_r{row}c{col}_tot_low{tot_low}_tot_high{tot_high}_toa_low{toa_low}_toa_high{toa_high}.png')

for toa_low in toa_lows:
    for toa_high in toa_highs:
        #Only cutting on toa
        plt.clf()
        toa_mask = (px_toa_codes>toa_low) & (px_toa_codes<toa_high)
        h3 = Hist1D(px_dts[toa_mask],
                    bins=dt_bins,
                    label='toa cut events {} ({:.1f}%)'.format(ak.sum(toa_mask),100*ak.sum(toa_mask)/len(px_dts)))
        h3.plot()
        h3.fit('gaus',color='r',label='toa_cut')
        plt.title('Resolution vs cuts (toa cut)')
        plt.xlabel('dt')
        plt.ylabel('count')
        #plt.savefig(f'DESY_tot_toa_studies/images/dt_toa_cut_r{row}c{col}_toa_low{toa_low}_toa_high{toa_high}.png')

In [ ]:
f = plt.figure()
f.set_figwidth(9)
f.set_figheight(6)

row=5
col=6
px_mask = (rows==row) & (cols==col)

cal_median = np.median(cal_codes[px_mask])# & data_mask])
cal_mask = (cal_codes>cal_median-2) & (cal_codes<cal_median+2)
px_tot_codes = tot_codes[px_mask & cal_mask]
px_toa_codes = toa_codes[px_mask & cal_mask]

toa_bins = np.linspace(0,800,80)
px_mask = (rows==row) & (cols==col)
tot_mask = (px_tot_codes>tot_low) & (px_tot_codes<tot_high)
toa_mask = (px_toa_codes>toa_low) & (px_toa_codes<toa_high)
h1 = Hist1D(toa_codes[px_mask],bins=toa_bins,label=f'no cut total events: {ak.sum(px_mask)}')
h2 = Hist1D(px_toa_codes, bins=toa_bins,label=f'cal cut total events: {len(px_toa_codes)}')
#h2 = Hist1D(toa_codes[tot_mask], bins=toa_bins,label=f'tot cut ({tot_low}-{tot_high}) total events: {ak.sum(tot_mask)}')
#h2 = Hist1D(
#h3 = Hist1D(toa_codes[toa_mask], bins=toa_bins,label=f'toa cut ({toa_low}) total events: {ak.sum(toa_mask)}')

h1.plot()
h2.plot()
#h3.plot()
#plt.axvline(30,color='r')

plt.title(f'toa with tot cuts r:{row} c:{col}')
plt.xlabel('toa code')
plt.ylabel('count')

print(h1.restrict(0,30).integral)
print(h2.restrict(0,30).integral)

In [ ]:
f = plt.figure()
f.set_figwidth(9)
f.set_figheight(6)

row=9
col=8
px_mask = (rows==row) & (cols==col)

cal_mode = st.mode(np.array(cal_codes[px_mask]))[0]
cal_mask = (cal_codes>cal_mode-2) & (cal_codes<cal_mode+2)
px_tot_codes = tot_codes[px_mask & cal_mask]
px_toa_codes = toa_codes[px_mask & cal_mask]

toa_bins = np.linspace(0,800,80)
px_mask = (rows==row) & (cols==col)
tot_mask = (px_tot_codes>tot_low) & (px_tot_codes<tot_high)
toa_mask = (px_toa_codes>toa_low) & (px_toa_codes<toa_high)
h1 = Hist1D(toa_codes[px_mask],bins=toa_bins,label=f'no cut total events: {ak.sum(px_mask)}')
h2 = Hist1D(px_toa_codes, bins=toa_bins,label=f'cal cut total events: {len(px_toa_codes)}')

#h1.plot()
h2.plot()

plt.title(f'toa with tot cuts r:{row} c:{col}')
plt.xlabel('toa code')
plt.ylabel('count')
plt.axvline(toa_low,color='r')

#print(h1.restrict(0,30).integral)
print(f'Count above {toa_low}: {h2.restrict(toa_low,700).integral}')

In [ ]:
f = plt.figure()
f.set_figwidth(9)
f.set_figheight(6)

row=9
col=8

toa_bins = np.linspace(0,800,80)
px_mask = (rows==row) & (cols==col)

cal_mode = st.mode(np.array(cal_codes[px_mask]))[0]
cal_mask = (cal_codes>cal_mode-2) & (cal_codes<cal_mode+2)
px_tot_codes = tot_codes[px_mask & cal_mask]
px_toa_codes = toa_codes[px_mask & cal_mask]
toa_mask = px_toa_codes > toa_low
px_dts = dts[px_mask & cal_mask]

h1 = Hist1D(px_dts[px_toa_codes>300],
                bins=dt_bins,
                label='tot cut events {} ({:.1f}%)'.format(ak.sum(toa_mask),100*ak.sum(toa_mask)/len(px_dts)))
h1.plot()
h1.fit('gaus',color='r',label=f'toa above {toa_low}')
plt.title(f'Resolution vs cuts (toa above {toa_low})')
plt.xlabel('dt')
plt.ylabel('count')

In [ ]:
f = plt.figure()
f.set_figwidth(9)
f.set_figheight(6)

row=9
col=8

toa_bins = np.linspace(0,800,80)
px_mask = (rows==row) & (cols==col)

cal_mode = st.mode(np.array(cal_codes[px_mask]))[0]
cal_mask = (cal_codes>cal_mode-2) & (cal_codes<cal_mode+2)
px_tot_codes = tot_codes[px_mask & cal_mask]
px_toa_codes = toa_codes[px_mask & cal_mask]
toa_mask = px_toa_codes < toa_low
px_dts = dts[px_mask & cal_mask]

h1 = Hist1D(px_dts[px_toa_codes<tot_low],
                bins=dt_bins,
                label='tot cut events {} ({:.1f}%)'.format(ak.sum(toa_mask),100*ak.sum(toa_mask)/len(px_dts)))
h1.plot()
h1.fit('gaus',color='r',label=f'toa below {toa_low}')
plt.title(f'Resolution vs cuts (toa below {toa_low})')
plt.xlabel('dt')
plt.ylabel('count')

In [ ]:
splits = [np.linspace(0,500,50),np.linspace(0,800,50)]
h1 = Hist2D((tot_codes,toa_codes), bins=splits)
h1.plot()
plt.title('No cal cut')
plt.xlabel('tot code')
plt.ylabel('toa code')
plt.axvline(50,color='r')
plt.axvline(200,color='r')
plt.axhline(20,color='g')
plt.axhline(600,color='g')

In [ ]:
splits = [np.linspace(0,500,50),np.linspace(0,1200,50)]
h1 = Hist2D((tot_codes,cal_codes), bins=splits)
h1.plot()
plt.xlabel('tot code')
plt.ylabel('cal code')
plt.axvline(50,color='r')
plt.axvline(200,color='r')


In [ ]:
splits = [np.linspace(0,1200,50),np.linspace(0,800,50)]
h1 = Hist2D((cal_codes,toa_codes), bins=splits)
h1.plot()
plt.xlabel('cal code')
plt.ylabel('toa code')
plt.axhline(20,color='g')
plt.axhline(600,color='g')

In [ ]:
cal_mode = st.mode(np.array(cal_codes))[0]


In [ ]:
cal_mode

In [ ]:
cal_mod_mask = (cal_codes < (cal_mode+2)) & (cal_codes > (cal_mode-2))

In [ ]:
splits = [np.linspace(0,500,50),np.linspace(0,800,50)]
h1 = Hist2D((tot_codes[cal_mod_mask],toa_codes[cal_mod_mask]), bins=splits)
h1.plot()
plt.title('with cal cut')
plt.xlabel('tot code')
plt.ylabel('toa code')
plt.axvline(50,color='r')
plt.axvline(200,color='r')
plt.axhline(20,color='g')
plt.axhline(600,color='g')